In [ ]:
import util

from datetime import datetime, timedelta
import pytz

cities = {"stockholm":[59.3294,18.0687], "goteborg":[57.7072,11.9668]}
keys = list(cities.keys())
    
project = util.connect_to_hopsworks_project()

fs = util.create_fs(project)

model, fv = util.get_model(project)
starttime=first_hour = (
    datetime.now(pytz.timezone("Europe/Stockholm"))
    .replace(hour=0, minute=0, second=0, microsecond=0)
    - timedelta(days=0) 
).astimezone(pytz.UTC)
combined_data = util.get_forecast_weather(fs, starttime, keys[0], keys[1])

batch_data, first_hour, first_roll = util.add_first_price_features(fs, combined_data)
first_roll = first_roll.sort_values("date").reset_index(drop=True)
pred_data = util.predictions(first_hour, batch_data, model, first_roll)
print(len(pred_data), pred_data)
    
model_dir,images_dir = util.create_dirs()
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #    print(batch_data)
forecast_path = util.plot_price_forecast("SE3",batch_data, images_dir + "/forecast.png", False)

monitor_fg = util.monitoring_fg(batch_data,fs)

monitoring_df = util.monitor_df(monitor_fg)

elec_fg, elec_df = util.elprice_pred_df(fs)

hindcast_df = util.hindcast(elec_df,monitoring_df)

hindcast_path = util.plot_price_forecast("SE3",hindcast_df, images_dir + "/hindcast.png", True)
today = datetime.now()
util.upload_to_hops(project,today, forecast_path,hindcast_path)